# Prepping data

In [2]:
# file imports
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.optimize import minimize as minimize
from itertools import islice

# splitting data in train test and val set
def data_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)  
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    return X_train, X_test, X_val, y_train, y_test, y_val

data = pd.read_excel(r'C:\Users\RE-Giorgio\Downloads\dataPoints.xlsx')
X = np.array(data.iloc[:,:2])
y = np.array(data.iloc[:, 2])
X_train, X_test, X_val, y_train, y_test, y_val = data_split(X, y)

# Neural Network parent class

In [19]:
class NeuralNetwork:
    
    def __init__(self, X, y, N, sigma):

        self.X = X
        self.y = y
        self.N = N
        self.bias = np.zeros(self.N)
        self.weights1 = np.random.normal(0,1,(self.X.shape[1], self.N))
        self.weights2 = np.random.normal(0,1,(self.N, 1))
        self.output = np.zeros(y.shape[0])
        self.rho = np.random.normal(10e-4,10e-5,1)
        self.sigma = sigma
    
    # needed to apply the norm on the parameters' vectors all togheter for the regularization
    def concatenate(self,l):
        l = [np.array(array)for array in l]
        return np.concatenate(l)
    

    def loss(self,params, fixed_params):
        
        # the next 2 lines are two divide the concatenated vectors for the forward propagation
        seclist = [self.X.shape[1]*self.N, self.N, self.N]
        weights1, weights2, bias = [list(islice(iter(params), 0, i)) for i in seclist]
        y, rho, sigma = fixed_params
        return 0.5 * np.mean(np.square((self.act_fun(self.forward_propagation()) - y))) +\
            rho*np.square(np.linalg.norm(self.concatenate([weights1, weights2, bias])))

## MLP child class

In [20]:
class Mlp(NeuralNetwork):
    
    def act_fun(self,x):
        return (1-np.exp(-2*x*self.sigma))/(1+np.exp(-2*x*self.sigma))

    def forward_propagation(self):
        
        self.hidden_layer = self.act_fun(np.dot(self.X, self.weights1))
        self.output = np.dot(self.hidden_layer, self.weights2)
        return self.output
                          
    def optimization(self):
        function_args = np.array([self.y, self.rho, self.sigma])
        inits = self.concatenate([self.weights1, self.weights2, self.bias])
        return minimize(self.loss, x0 = inits, method='CG', args = function_args)
        

## first run

In [18]:
nn = Mlp(X_train, y_train, 10, 1)
nn.optimization()

[array([[-0.97724178,  0.97186897, -1.32860957, -1.34163452, -0.62986345,
         1.39193135,  3.34354422, -0.84235043,  0.35305372,  0.58121186],
       [ 1.36682062,  0.31838551,  0.75894287, -0.20840648, -1.25117233,
        -0.70891522, -0.26394423, -0.59336863, -0.46684087, -0.0532454 ]]), array([[ 1.45180648],
       [-0.50920433],
       [ 1.87928975],
       [ 1.67104296],
       [ 1.51489989],
       [ 0.51956168],
       [-1.06586428],
       [-0.37866888],
       [-3.24642061],
       [ 1.34143297]]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]
[[-0.9772417816418003, 0.9718689675444198, -1.3286095675703042, -1.3416345189062504, -0.6298634483734209, 1.3919313484555265, 3.343544219830768, -0.8423504271937045, 0.3530537234460802, 0.5812118579020843, 1.3668206221496888, 0.31838551476759946, 0.7589428700622524, -0.20840647652611444, -1.2511723349958708, -0.7089152195607225, -0.26394423128928984, -0.5933686297583196, -0.4668408724866907, -0.05324539994032893], [-0.9772417816

[[0.01869021785749414, -0.01858864199417409, 0.025411768465127545, 0.025661864060271978, 0.012047172275697307, -0.02662291198820342, -0.06395130695607468, 0.016112738173018788, -0.006753614548557415, -0.011117385893743333, 0.8945780532126195, 0.20838087419632892, 0.4967236804098175, -0.13640192370863052, -0.818885396913248, -0.4639819677163986, -0.17274955788034102, -0.38835775809117234, -0.3055458290436119, -0.034848911390226306], [0.01869021785749414, -0.01858864199417409, 0.025411768465127545, 0.025661864060271978, 0.012047172275697307, -0.02662291198820342, -0.06395130695607468, 0.016112738173018788, -0.006753614548557415, -0.011117385893743333], [0.01869021785749414, -0.01858864199417409, 0.025411768465127545, 0.025661864060271978, 0.012047172275697307, -0.02662291198820342, -0.06395130695607468, 0.016112738173018788, -0.006753614548557415, -0.011117385893743333]]
[[0.01869021785749414, -0.01858864199417409, 0.025411768465127545, 0.025661864060271978, 0.012047157374536113, -0.0266

[[0.0187230000356694, -0.018621237973010917, 0.025456345604042595, 0.025706888285963538, 0.01206828666185017, -0.02666960402891, -0.06406348211804469, 0.016141008591690903, -0.006765463508581461, -0.011136889121831835, 0.8961473176408381, 0.20874643514790855, 0.49759503993284415, -0.136641191123792, -0.8203218819972311, -0.46479588368256686, -0.17305258752023872, -0.3890390160105611, -0.3060818074307932, -0.034910045787914394], [0.0187230000356694, -0.018621237973010917, 0.025456345604042595, 0.025706888285963538, 0.01206828666185017, -0.02666960402891, -0.06406348211804469, 0.016141008591690903, -0.006765463508581461, -0.011136889121831835], [0.0187230000356694, -0.018621237973010917, 0.025456345604042595, 0.025706888285963538, 0.01206828666185017, -0.02666960402891, -0.06406348211804469, 0.016141008591690903, -0.006765463508581461, -0.011136889121831835]]
[[0.0187230000356694, -0.018621237973010917, 0.025456345604042595, 0.025706888285963538, 0.01206828666185017, -0.02666960402891, -

[[0.0020696535226367857, -0.002062480723903727, 0.0028111590351982477, 0.0028345816346505223, 0.0013346087063096854, -0.0029500473499685723, -0.007078499837277583, 0.0017796359062567692, -0.0007461918163658571, -0.0012292492528723706, 0.09896098810579412, 0.02304902663416944, 0.054944402235314105, -0.015093344221764268, -0.09058745933382217, -0.051326572869095866, -0.019113530452205485, -0.042959992961074056, -0.03380478674267867, -0.0038537717623680495], [0.0020696535226367857, -0.002062480723903727, 0.0028111590351982477, 0.0028345816346505223, 0.0013346087063096854, -0.0029500473499685723, -0.007078499837277583, 0.0017796359062567692, -0.0007461918163658571, -0.0012292492528723706], [0.0020696535226367857, -0.002062480723903727, 0.0028111590351982477, 0.0028345816346505223, 0.0013346087063096854, -0.0029500473499685723, -0.007078499837277583, 0.0017796359062567692, -0.0007461918163658571, -0.0012292492528723706]]
[[0.0020696535226367857, -0.002062480723903727, 0.0028111590351982477,

[[-0.001377297985885248, 0.0013746211170691733, -0.0018676118645743964, -0.0018829620865485133, -0.0008859176313139108, 0.001965503001174311, 0.004711567120196172, -0.0011826553344353695, 0.0004993068408607054, 0.0008212548355294239, 0.03990695613767736, 0.009294018055868963, 0.022159433620735858, -0.006086933384590806, -0.03652710450498764, -0.020694941256318156, -0.007704560692264384, -0.017321725563193502, -0.013628778398982078, -0.0015502608580559157], [-0.001377297985885248, 0.0013746211170691733, -0.0018676118645743964, -0.0018829620865485133, -0.0008859176313139108, 0.001965503001174311, 0.004711567120196172, -0.0011826553344353695, 0.0004993068408607054, 0.0008212548355294239], [-0.001377297985885248, 0.0013746211170691733, -0.0018676118645743964, -0.0018829620865485133, -0.0008859176313139108, 0.001965503001174311, 0.004711567120196172, -0.0011826553344353695, 0.0004993068408607054, 0.0008212548355294239]]
[[-0.001377297985885248, 0.0013746211170691733, -0.0018676118645743964,

[[-0.000853434323831142, 0.0008555258265970986, -0.0011522856932829663, -0.0011643657903109954, -0.0005481488082607205, 0.0012214232944978373, 0.0029238993600167527, -0.0007323868526993827, 0.0003144105991685446, 0.0005113964198550866, 0.007228021526311848, 0.0016836328307352926, 0.004011010805225266, -0.0011021661687184402, -0.00660987831573214, -0.003747400891568642, -0.0013971257275283235, -0.003136193269881192, -0.002468197908967519, -0.0002810186503821006], [-0.000853434323831142, 0.0008555258265970986, -0.0011522856932829663, -0.0011643657903109954, -0.0005481488082607205, 0.0012214232944978373, 0.0029238993600167527, -0.0007323868526993827, 0.0003144105991685446, 0.0005113964198550866], [-0.000853434323831142, 0.0008555258265970986, -0.0011522856932829663, -0.0011643657903109954, -0.0005481488082607205, 0.0012214232944978373, 0.0029238993600167527, -0.0007323868526993827, 0.0003144105991685446, 0.0005113964198550866]]
[[-0.000853434323831142, 0.0008555109254359047, -0.0011522707

     fun: 1.917660236411037
     jac: array([ 1.98185444e-06, -1.99675560e-06,  2.66730785e-06,  2.71201134e-06,
        1.26659870e-06, -2.83122063e-06, -6.80983067e-06,  1.71363354e-06,
       -7.45058060e-07, -1.17719173e-06,  3.51667404e-06,  8.19563866e-07,
        1.95205212e-06, -5.36441803e-07, -3.21865082e-06, -1.81794167e-06,
       -6.85453415e-07, -1.51991844e-06, -1.20699406e-06, -1.34110451e-07,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 1764
     nit: 6
    njev: 42
  status: 0
 success: True
       x: array([ 3.86089129e-04, -3.88952930e-04,  5.21160664e-04,  5.31086947e-04,
        2